In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
import malaya_speech
import malaya_speech.train
from malaya_speech.train.model import melgan
import malaya_speech.config
import numpy as np

In [4]:
melgan_config = malaya_speech.config.melgan_config

In [5]:
generator = melgan.Generator(
    melgan.config.GeneratorConfig(**melgan_config['melgan_generator_params']),
    name='melgan-generator',
)
discriminator = melgan.MultiScaleDiscriminator(
    melgan.config.DiscriminatorConfig(**melgan_config['melgan_discriminator_params']),
    name='melgan-discriminator',
)

In [6]:
mels_loss = melgan.loss.TFMelSpectrogram()

In [7]:
y = tf.placeholder(tf.float32, (None, None))
x = tf.placeholder(tf.float32, (None, None, 80))
y_hat = generator(x)
d_hat = discriminator(y_hat)
d = discriminator(tf.expand_dims(y, -1))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
spect_loss = malaya_speech.train.loss.calculate_2d_loss(
    y, tf.squeeze(y_hat, -1), loss_fn = mels_loss
)

In [9]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [11]:
# y_hat_, loss, d_, d_hat_ = sess.run([y_hat, spect_loss, d_hat, d], 
#                                     feed_dict = {x: np.random.uniform(size=(1,200,80)),
#                                                  y: np.random.uniform(size=(1,51200))})